transpose 연습

In [27]:
import numpy as np
arr = np.arange(1, 3*3*3 + 1).reshape(3, 3, 3)
arr

array([[[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9]],

       [[10, 11, 12],
        [13, 14, 15],
        [16, 17, 18]],

       [[19, 20, 21],
        [22, 23, 24],
        [25, 26, 27]]])

In [ ]:
arr.transpose(0, 2, 1) # 일반적인 전치행렬

array([[[ 1,  4,  7],
        [ 2,  5,  8],
        [ 3,  6,  9]],

       [[10, 13, 16],
        [11, 14, 17],
        [12, 15, 18]],

       [[19, 22, 25],
        [20, 23, 26],
        [21, 24, 27]]])

In [34]:
'''
[1, 2, 3]
[10, 11, 12]
[19, 20, 21]

[4, 5, 6]
[13, 14, 15]
[22, 23, 24]

[7, 8, 9]
[16, 17, 18]
[25, 26, 27]
'''
arr.transpose(1, 0, 2)

array([[[ 1,  2,  3],
        [10, 11, 12],
        [19, 20, 21]],

       [[ 4,  5,  6],
        [13, 14, 15],
        [22, 23, 24]],

       [[ 7,  8,  9],
        [16, 17, 18],
        [25, 26, 27]]])

In [35]:
'''
[1, 10, 19]
[2, 11, 20]
[3, 12, 21]

[4, 13, 22]
[5, 14, 23]
[6, 15, 24]

[7, 16, 25]
[8, 17, 26]
[9, 18, 27]
'''
arr.transpose(1, 2, 0)

array([[[ 1, 10, 19],
        [ 2, 11, 20],
        [ 3, 12, 21]],

       [[ 4, 13, 22],
        [ 5, 14, 23],
        [ 6, 15, 24]],

       [[ 7, 16, 25],
        [ 8, 17, 26],
        [ 9, 18, 27]]])

In [ ]:
'''
[1, 4, 7]
[10, 11, 12]
[19, 20, 21]

[2, 5, 8]
[11, 14, 17]
[20, 23, 26]

[3, 6, 9]
[12, 15, 18]
[21, 24, 27]
'''
arr.transpose(2, 0, 1)

array([[[ 1,  4,  7],
        [10, 13, 16],
        [19, 22, 25]],

       [[ 2,  5,  8],
        [11, 14, 17],
        [20, 23, 26]],

       [[ 3,  6,  9],
        [12, 15, 18],
        [21, 24, 27]]])

In [36]:
'''
[1, 10, 19]
[4, 13, 22]
[7, 16, 25]

[2, 11, 20]
[5, 14, 23]
[8, 17, 26]

[3, 12, 21]
[6, 15, 24]
[9, 81, 27]
'''
arr.transpose(2, 1, 0)

array([[[ 1, 10, 19],
        [ 4, 13, 22],
        [ 7, 16, 25]],

       [[ 2, 11, 20],
        [ 5, 14, 23],
        [ 8, 17, 26]],

       [[ 3, 12, 21],
        [ 6, 15, 24],
        [ 9, 18, 27]]])

In [39]:
brr = np.arange(1, 2*3*3*3+1).reshape(2, 3, 3, 3)
brr

array([[[[ 1,  2,  3],
         [ 4,  5,  6],
         [ 7,  8,  9]],

        [[10, 11, 12],
         [13, 14, 15],
         [16, 17, 18]],

        [[19, 20, 21],
         [22, 23, 24],
         [25, 26, 27]]],


       [[[28, 29, 30],
         [31, 32, 33],
         [34, 35, 36]],

        [[37, 38, 39],
         [40, 41, 42],
         [43, 44, 45]],

        [[46, 47, 48],
         [49, 50, 51],
         [52, 53, 54]]]])

In [41]:
'''
[1, 4, 7]
[10, 13, 16]
[19, 22, 25]

[2, 5, 8]
[11, 14, 17]
[20, 23, 26]

[3, 6, 9]
[12, 15, 18]
[21, 24, 27]


[28, 31, 34]
[37, 40, 43]
[46, 49, 52]

[29, 32, 35]
[38, 41, 44]
[47, 50, 53]

[30, 33, 36]
[39, 42, 45]
[48, 51, 54]
'''
brr.transpose(0, 3, 1, 2)

array([[[[ 1,  4,  7],
         [10, 13, 16],
         [19, 22, 25]],

        [[ 2,  5,  8],
         [11, 14, 17],
         [20, 23, 26]],

        [[ 3,  6,  9],
         [12, 15, 18],
         [21, 24, 27]]],


       [[[28, 31, 34],
         [37, 40, 43],
         [46, 49, 52]],

        [[29, 32, 35],
         [38, 41, 44],
         [47, 50, 53]],

        [[30, 33, 36],
         [39, 42, 45],
         [48, 51, 54]]]])

In [42]:
brr.transpose(0, 3, 1, 2).reshape(2*3, 3*3)

array([[ 1,  4,  7, 10, 13, 16, 19, 22, 25],
       [ 2,  5,  8, 11, 14, 17, 20, 23, 26],
       [ 3,  6,  9, 12, 15, 18, 21, 24, 27],
       [28, 31, 34, 37, 40, 43, 46, 49, 52],
       [29, 32, 35, 38, 41, 44, 47, 50, 53],
       [30, 33, 36, 39, 42, 45, 48, 51, 54]])

In [40]:
'''
[1, 4, 7]
[28, 31, 34]

[10, 13, 16]
[37, 40, 43]

[19, 22, 25]
[46, 49, 52]


[2, 5, 8]
[29, 32, 35]

[11, 14, 17]
[38, 41, 44]

[20, 23, 26]
[47, 50, 53]


[3, 6, 9]
[30, 33, 36]

[12, 15, 18]
[39, 42, 45]

[21, 24, 27]
[48, 51, 54]
'''
brr.transpose(3, 1, 0, 2) # shape: (3, 3, 2, 3)

array([[[[ 1,  4,  7],
         [28, 31, 34]],

        [[10, 13, 16],
         [37, 40, 43]],

        [[19, 22, 25],
         [46, 49, 52]]],


       [[[ 2,  5,  8],
         [29, 32, 35]],

        [[11, 14, 17],
         [38, 41, 44]],

        [[20, 23, 26],
         [47, 50, 53]]],


       [[[ 3,  6,  9],
         [30, 33, 36]],

        [[12, 15, 18],
         [39, 42, 45]],

        [[21, 24, 27],
         [48, 51, 54]]]])

## numpy에서 for 문을 사용하면 성능이 떨어짐!

## for문 대신에 im2col로 데이터 전개하기

In [ ]:
# im2col 함수에서 행렬곱을 하여 convolution을 바로 구현하지 않고
# im2col 과 transpose된 벡터(커널)를 행렬곱 하는 이유는 backpropagation을 효율적으로 구현하기 위해!! 행렬곱은 미분이 되므로!!
# 강사님은 for loop는 미분이 없다라고 하셨는데 for loop는 미분하기에 효율적이지 않음!!
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화).
    
    Parameters
    ----------
    input_data : 4차원 배열 형태의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    col : 2차원 배열
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant') # N, C: 패딩없음 (0, 0)
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1) # (N, out_h, out_w, C, filter_h, filter_w)
    return col

In [22]:
x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(9, 75)
(90, 75)


# np.pad 확인

In [ ]:
input_data = np.array([[[[1, 2], 
                         [3, 4]],
                        [[5, 6],
                         [7, 8]]],
                       [[[9, 10],
                         [11, 12]],
                        [[13, 14],
                         [15, 16]]]])
print(input_data)
pad = 1
np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')

[[[[ 1  2]
   [ 3  4]]

  [[ 5  6]
   [ 7  8]]]


 [[[ 9 10]
   [11 12]]

  [[13 14]
   [15 16]]]]


array([[[[ 0,  0,  0,  0],
         [ 0,  1,  2,  0],
         [ 0,  3,  4,  0],
         [ 0,  0,  0,  0]],

        [[ 0,  0,  0,  0],
         [ 0,  5,  6,  0],
         [ 0,  7,  8,  0],
         [ 0,  0,  0,  0]]],


       [[[ 0,  0,  0,  0],
         [ 0,  9, 10,  0],
         [ 0, 11, 12,  0],
         [ 0,  0,  0,  0]],

        [[ 0,  0,  0,  0],
         [ 0, 13, 14,  0],
         [ 0, 15, 16,  0],
         [ 0,  0,  0,  0]]]])

In [2]:
input_data = np.array([[[[1, 2], 
                         [3, 4]],
                        [[5, 6],
                         [7, 8]]],
                       [[[9, 10],
                         [11, 12]],
                        [[13, 14],
                         [15, 16]]]])
print(input_data)
pad = 1
np.pad(input_data, [(0,0), (0,0), (0, 0), (0, pad)], 'constant')

[[[[ 1  2]
   [ 3  4]]

  [[ 5  6]
   [ 7  8]]]


 [[[ 9 10]
   [11 12]]

  [[13 14]
   [15 16]]]]


array([[[[ 1,  2,  0],
         [ 3,  4,  0]],

        [[ 5,  6,  0],
         [ 7,  8,  0]]],


       [[[ 9, 10,  0],
         [11, 12,  0]],

        [[13, 14,  0],
         [15, 16,  0]]]])

In [17]:
input_data = np.array([[[[1, 2], 
                         [3, 4]],
                        [[5, 6],
                         [7, 8]]],
                       [[[9, 10],
                         [11, 12]],
                        [[13, 14],
                         [15, 16]]]])
pad = 1
np.pad(input_data, [(pad, 0), (0, 0), (0,0), (0,0)], 'constant')

array([[[[ 0,  0],
         [ 0,  0]],

        [[ 0,  0],
         [ 0,  0]]],


       [[[ 1,  2],
         [ 3,  4]],

        [[ 5,  6],
         [ 7,  8]]],


       [[[ 9, 10],
         [11, 12]],

        [[13, 14],
         [15, 16]]]])

In [35]:
input_data = np.array([[[[1, 2], 
                         [3, 4]],
                        [[5, 6],
                         [7, 8]]],
                       [[[9, 10],
                         [11, 12]],
                        [[13, 14],
                         [15, 16]]]])
pad = 1
np.pad(input_data, [(0, 0), (0, pad), (0,0), (0,0)], 'constant')

array([[[[ 1,  2],
         [ 3,  4]],

        [[ 5,  6],
         [ 7,  8]],

        [[ 0,  0],
         [ 0,  0]]],


       [[[ 9, 10],
         [11, 12]],

        [[13, 14],
         [15, 16]],

        [[ 0,  0],
         [ 0,  0]]]])

# im2col 작동 원리 디버깅

---
img shape: 2, 2, 2, 2, filter shape: 2, 1, 1, stride: 1

In [92]:
input_data = np.array([[[[1, 2], 
                         [3, 4]],
                        [[5, 6],
                         [7, 8]]],
                       [[[9, 10],
                         [11, 12]],
                        [[13, 14],
                         [15, 16]]]])


N, C, H, W = input_data.shape
filter_h = filter_w = 1
pad = 0
stride = 1
out_h = (H + 2*pad - filter_h)//stride + 1
out_w = (W + 2*pad - filter_w)//stride + 1

img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant') # N, C: 패딩없음 (0, 0)
col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

for y in range(filter_h):
    y_max = y + stride*out_h
    for x in range(filter_w):
        x_max = x + stride*out_w
        col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

In [44]:
img

array([[[[ 1,  2],
         [ 3,  4]],

        [[ 5,  6],
         [ 7,  8]]],


       [[[ 9, 10],
         [11, 12]],

        [[13, 14],
         [15, 16]]]])

In [45]:
col

array([[[[[[ 1.,  2.],
           [ 3.,  4.]]]],



        [[[[ 5.,  6.],
           [ 7.,  8.]]]]],




       [[[[[ 9., 10.],
           [11., 12.]]]],



        [[[[13., 14.],
           [15., 16.]]]]]])

In [93]:
col.transpose(0, 4, 5, 1, 2, 3)

array([[[[[[ 1.]],

          [[ 5.]]],


         [[[ 2.]],

          [[ 6.]]]],



        [[[[ 3.]],

          [[ 7.]]],


         [[[ 4.]],

          [[ 8.]]]]],




       [[[[[ 9.]],

          [[13.]]],


         [[[10.]],

          [[14.]]]],



        [[[[11.]],

          [[15.]]],


         [[[12.]],

          [[16.]]]]]])

In [46]:
col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1) # (N*out_h*out_w, C*filter_h*filter_w)
col

array([[ 1.,  5.],
       [ 2.,  6.],
       [ 3.,  7.],
       [ 4.,  8.],
       [ 9., 13.],
       [10., 14.],
       [11., 15.],
       [12., 16.]])

---
img size: 1, 1, 3, 3, filter size: 1, 2, 2, stride: 1

In [81]:
input_data = np.arange(1, 1*1*4*4 + 1).reshape(1, 1, 4, 4)
input_data

array([[[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12],
         [13, 14, 15, 16]]]])

In [94]:
import numpy as np
input_data = np.arange(1, 1*1*4*4 + 1).reshape(1, 1, 4, 4)

N, C, H, W = input_data.shape
filter_h = filter_w = 2
pad = 0
stride = 1
out_h = (H + 2*pad - filter_h)//stride + 1
out_w = (W + 2*pad - filter_w)//stride + 1

img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant') # N, C: 패딩없음 (0, 0)
col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

for y in range(filter_h):
    y_max = y + stride*out_h
    for x in range(filter_w):
        x_max = x + stride*out_w
        col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

In [83]:
col

array([[[[[[ 1.,  2.,  3.],
           [ 5.,  6.,  7.],
           [ 9., 10., 11.]],

          [[ 2.,  3.,  4.],
           [ 6.,  7.,  8.],
           [10., 11., 12.]]],


         [[[ 5.,  6.,  7.],
           [ 9., 10., 11.],
           [13., 14., 15.]],

          [[ 6.,  7.,  8.],
           [10., 11., 12.],
           [14., 15., 16.]]]]]])

In [95]:
col.transpose(1, 2, 3, 0, 4, 5)

array([[[[[[ 1.,  2.,  3.],
           [ 5.,  6.,  7.],
           [ 9., 10., 11.]]],


         [[[ 2.,  3.,  4.],
           [ 6.,  7.,  8.],
           [10., 11., 12.]]]],



        [[[[ 5.,  6.,  7.],
           [ 9., 10., 11.],
           [13., 14., 15.]]],


         [[[ 6.,  7.,  8.],
           [10., 11., 12.],
           [14., 15., 16.]]]]]])

In [87]:
col.transpose(1, 2, 3, 0, 4, 5).reshape(C*filter_h*filter_w, -1)

array([[ 1.,  2.,  3.,  5.,  6.,  7.,  9., 10., 11.],
       [ 2.,  3.,  4.,  6.,  7.,  8., 10., 11., 12.],
       [ 5.,  6.,  7.,  9., 10., 11., 13., 14., 15.],
       [ 6.,  7.,  8., 10., 11., 12., 14., 15., 16.]])

---
img size: 4, 3, 4, 4, filter size: 3, 3, 3, stride: 1

In [ ]:
input_data = np.arange(1, 4*3*4*4 + 1).reshape(4, 3, 4, 4)
input_data

array([[[[  1,   2,   3,   4],
         [  5,   6,   7,   8],
         [  9,  10,  11,  12],
         [ 13,  14,  15,  16]],

        [[ 17,  18,  19,  20],
         [ 21,  22,  23,  24],
         [ 25,  26,  27,  28],
         [ 29,  30,  31,  32]],

        [[ 33,  34,  35,  36],
         [ 37,  38,  39,  40],
         [ 41,  42,  43,  44],
         [ 45,  46,  47,  48]]],


       [[[ 49,  50,  51,  52],
         [ 53,  54,  55,  56],
         [ 57,  58,  59,  60],
         [ 61,  62,  63,  64]],

        [[ 65,  66,  67,  68],
         [ 69,  70,  71,  72],
         [ 73,  74,  75,  76],
         [ 77,  78,  79,  80]],

        [[ 81,  82,  83,  84],
         [ 85,  86,  87,  88],
         [ 89,  90,  91,  92],
         [ 93,  94,  95,  96]]],


       [[[ 97,  98,  99, 100],
         [101, 102, 103, 104],
         [105, 106, 107, 108],
         [109, 110, 111, 112]],

        [[113, 114, 115, 116],
         [117, 118, 119, 120],
         [121, 122, 123, 124],
         [125, 126, 1

In [100]:
import numpy as np
input_data = np.arange(1, 4*3*4*4 + 1).reshape(4, 3, 4, 4)

N, C, H, W = input_data.shape
filter_h = filter_w = 3
pad = 0
stride = 1
out_h = (H + 2*pad - filter_h)//stride + 1
out_w = (W + 2*pad - filter_w)//stride + 1

img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant') # N, C: 패딩없음 (0, 0)
col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

for y in range(filter_h):
    y_max = y + stride*out_h
    for x in range(filter_w):
        x_max = x + stride*out_w
        col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

In [48]:
col

array([[[[[[  1.,   2.],
           [  5.,   6.]],

          [[  2.,   3.],
           [  6.,   7.]],

          [[  3.,   4.],
           [  7.,   8.]]],


         [[[  5.,   6.],
           [  9.,  10.]],

          [[  6.,   7.],
           [ 10.,  11.]],

          [[  7.,   8.],
           [ 11.,  12.]]],


         [[[  9.,  10.],
           [ 13.,  14.]],

          [[ 10.,  11.],
           [ 14.,  15.]],

          [[ 11.,  12.],
           [ 15.,  16.]]]],



        [[[[ 17.,  18.],
           [ 21.,  22.]],

          [[ 18.,  19.],
           [ 22.,  23.]],

          [[ 19.,  20.],
           [ 23.,  24.]]],


         [[[ 21.,  22.],
           [ 25.,  26.]],

          [[ 22.,  23.],
           [ 26.,  27.]],

          [[ 23.,  24.],
           [ 27.,  28.]]],


         [[[ 25.,  26.],
           [ 29.,  30.]],

          [[ 26.,  27.],
           [ 30.,  31.]],

          [[ 27.,  28.],
           [ 31.,  32.]]]],



        [[[[ 33.,  34.],
           [ 37.,  38.]

In [101]:
col.transpose(0, 4, 5, 1, 2, 3)


array([[[[[[  1.,   2.,   3.],
           [  5.,   6.,   7.],
           [  9.,  10.,  11.]],

          [[ 17.,  18.,  19.],
           [ 21.,  22.,  23.],
           [ 25.,  26.,  27.]],

          [[ 33.,  34.,  35.],
           [ 37.,  38.,  39.],
           [ 41.,  42.,  43.]]],


         [[[  2.,   3.,   4.],
           [  6.,   7.,   8.],
           [ 10.,  11.,  12.]],

          [[ 18.,  19.,  20.],
           [ 22.,  23.,  24.],
           [ 26.,  27.,  28.]],

          [[ 34.,  35.,  36.],
           [ 38.,  39.,  40.],
           [ 42.,  43.,  44.]]]],



        [[[[  5.,   6.,   7.],
           [  9.,  10.,  11.],
           [ 13.,  14.,  15.]],

          [[ 21.,  22.,  23.],
           [ 25.,  26.,  27.],
           [ 29.,  30.,  31.]],

          [[ 37.,  38.,  39.],
           [ 41.,  42.,  43.],
           [ 45.,  46.,  47.]]],


         [[[  6.,   7.,   8.],
           [ 10.,  11.,  12.],
           [ 14.,  15.,  16.]],

          [[ 22.,  23.,  24.],
           

In [102]:
r_col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1) # (N*out_h*out_w, C*filter_h*filter_w)
r_col # (4*2*2, 3*3*3) = (16, 27)

array([[  1.,   2.,   3.,   5.,   6.,   7.,   9.,  10.,  11.,  17.,  18.,
         19.,  21.,  22.,  23.,  25.,  26.,  27.,  33.,  34.,  35.,  37.,
         38.,  39.,  41.,  42.,  43.],
       [  2.,   3.,   4.,   6.,   7.,   8.,  10.,  11.,  12.,  18.,  19.,
         20.,  22.,  23.,  24.,  26.,  27.,  28.,  34.,  35.,  36.,  38.,
         39.,  40.,  42.,  43.,  44.],
       [  5.,   6.,   7.,   9.,  10.,  11.,  13.,  14.,  15.,  21.,  22.,
         23.,  25.,  26.,  27.,  29.,  30.,  31.,  37.,  38.,  39.,  41.,
         42.,  43.,  45.,  46.,  47.],
       [  6.,   7.,   8.,  10.,  11.,  12.,  14.,  15.,  16.,  22.,  23.,
         24.,  26.,  27.,  28.,  30.,  31.,  32.,  38.,  39.,  40.,  42.,
         43.,  44.,  46.,  47.,  48.],
       [ 49.,  50.,  51.,  53.,  54.,  55.,  57.,  58.,  59.,  65.,  66.,
         67.,  69.,  70.,  71.,  73.,  74.,  75.,  81.,  82.,  83.,  85.,
         86.,  87.,  89.,  90.,  91.],
       [ 50.,  51.,  52.,  54.,  55.,  56.,  58.,  59.,  60.,  66

In [72]:
col.transpose(1, 2, 3, 0, 4, 5).reshape(C*filter_h*filter_w, -1).transpose()

array([[  1.,   2.,   3.,   5.,   6.,   7.,   9.,  10.,  11.,  17.,  18.,
         19.,  21.,  22.,  23.,  25.,  26.,  27.,  33.,  34.,  35.,  37.,
         38.,  39.,  41.,  42.,  43.],
       [  2.,   3.,   4.,   6.,   7.,   8.,  10.,  11.,  12.,  18.,  19.,
         20.,  22.,  23.,  24.,  26.,  27.,  28.,  34.,  35.,  36.,  38.,
         39.,  40.,  42.,  43.,  44.],
       [  5.,   6.,   7.,   9.,  10.,  11.,  13.,  14.,  15.,  21.,  22.,
         23.,  25.,  26.,  27.,  29.,  30.,  31.,  37.,  38.,  39.,  41.,
         42.,  43.,  45.,  46.,  47.],
       [  6.,   7.,   8.,  10.,  11.,  12.,  14.,  15.,  16.,  22.,  23.,
         24.,  26.,  27.,  28.,  30.,  31.,  32.,  38.,  39.,  40.,  42.,
         43.,  44.,  46.,  47.,  48.],
       [ 49.,  50.,  51.,  53.,  54.,  55.,  57.,  58.,  59.,  65.,  66.,
         67.,  69.,  70.,  71.,  73.,  74.,  75.,  81.,  82.,  83.,  85.,
         86.,  87.,  89.,  90.,  91.],
       [ 50.,  51.,  52.,  54.,  55.,  56.,  58.,  59.,  60.,  66

---
img shape: 2, 3, 5, 5, filter shape: 1, 3, 3, 3, stride: 2

In [ ]:
input_data = np.arange(1, 2*3*5*5 + 1).reshape(2, 3, 5, 5)
input_data

array([[[[  1,   2,   3,   4,   5],
         [  6,   7,   8,   9,  10],
         [ 11,  12,  13,  14,  15],
         [ 16,  17,  18,  19,  20],
         [ 21,  22,  23,  24,  25]],

        [[ 26,  27,  28,  29,  30],
         [ 31,  32,  33,  34,  35],
         [ 36,  37,  38,  39,  40],
         [ 41,  42,  43,  44,  45],
         [ 46,  47,  48,  49,  50]],

        [[ 51,  52,  53,  54,  55],
         [ 56,  57,  58,  59,  60],
         [ 61,  62,  63,  64,  65],
         [ 66,  67,  68,  69,  70],
         [ 71,  72,  73,  74,  75]]],


       [[[ 76,  77,  78,  79,  80],
         [ 81,  82,  83,  84,  85],
         [ 86,  87,  88,  89,  90],
         [ 91,  92,  93,  94,  95],
         [ 96,  97,  98,  99, 100]],

        [[101, 102, 103, 104, 105],
         [106, 107, 108, 109, 110],
         [111, 112, 113, 114, 115],
         [116, 117, 118, 119, 120],
         [121, 122, 123, 124, 125]],

        [[126, 127, 128, 129, 130],
         [131, 132, 133, 134, 135],
         [136, 1

In [88]:
input_data = np.arange(1, 2*3*5*5 + 1).reshape(2, 3, 5, 5)

N, C, H, W = input_data.shape
filter_h = filter_w = 3
pad = 0
stride = 2
out_h = (H + 2*pad - filter_h)//stride + 1
out_w = (W + 2*pad - filter_w)//stride + 1

img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

y, x = 0, 2
y_max = y + stride * 2
x_max = x + stride * 2

col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride] # 2 번째 축 0으로 고정, 3 번째 축 2로 고정 
col.shape, col

((2, 3, 3, 3, 2, 2),
 array([[[[[[  0.,   0.],
            [  0.,   0.]],
 
           [[  0.,   0.],
            [  0.,   0.]],
 
           [[  3.,   5.],
            [ 13.,  15.]]],
 
 
          [[[  0.,   0.],
            [  0.,   0.]],
 
           [[  0.,   0.],
            [  0.,   0.]],
 
           [[  0.,   0.],
            [  0.,   0.]]],
 
 
          [[[  0.,   0.],
            [  0.,   0.]],
 
           [[  0.,   0.],
            [  0.,   0.]],
 
           [[  0.,   0.],
            [  0.,   0.]]]],
 
 
 
         [[[[  0.,   0.],
            [  0.,   0.]],
 
           [[  0.,   0.],
            [  0.,   0.]],
 
           [[ 28.,  30.],
            [ 38.,  40.]]],
 
 
          [[[  0.,   0.],
            [  0.,   0.]],
 
           [[  0.,   0.],
            [  0.,   0.]],
 
           [[  0.,   0.],
            [  0.,   0.]]],
 
 
          [[[  0.,   0.],
            [  0.,   0.]],
 
           [[  0.,   0.],
            [  0.,   0.]],
 
           [[  0.,   0.],


In [52]:
r_col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1) # (N*out_h*out_w, C*filter_h*filter_w)
r_col.shape, r_col # (1*2*2, 3*3*3) = (4, 27)

((8, 27),
 array([[  0.,   0.,   3.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          28.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  53.,   0.,
           0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   5.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          30.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  55.,   0.,
           0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,  13.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          38.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  63.,   0.,
           0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,  15.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          40.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  65.,   0.,
           0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,  78.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         103.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 128.,   0.,
           0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,  80.,   0.,   0.,  

In [90]:
for y in range(filter_h):
    for x in range(filter_w):
        y_max = y + stride * 2
        x_max = x + stride * 2

        col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]
col.shape, col

((2, 3, 3, 3, 2, 2),
 array([[[[[[  1.,   3.],
            [ 11.,  13.]],
 
           [[  2.,   4.],
            [ 12.,  14.]],
 
           [[  3.,   5.],
            [ 13.,  15.]]],
 
 
          [[[  6.,   8.],
            [ 16.,  18.]],
 
           [[  7.,   9.],
            [ 17.,  19.]],
 
           [[  8.,  10.],
            [ 18.,  20.]]],
 
 
          [[[ 11.,  13.],
            [ 21.,  23.]],
 
           [[ 12.,  14.],
            [ 22.,  24.]],
 
           [[ 13.,  15.],
            [ 23.,  25.]]]],
 
 
 
         [[[[ 26.,  28.],
            [ 36.,  38.]],
 
           [[ 27.,  29.],
            [ 37.,  39.]],
 
           [[ 28.,  30.],
            [ 38.,  40.]]],
 
 
          [[[ 31.,  33.],
            [ 41.,  43.]],
 
           [[ 32.,  34.],
            [ 42.,  44.]],
 
           [[ 33.,  35.],
            [ 43.,  45.]]],
 
 
          [[[ 36.,  38.],
            [ 46.,  48.]],
 
           [[ 37.,  39.],
            [ 47.,  49.]],
 
           [[ 38.,  40.],


In [91]:
col.transpose(0, 4, 5, 1, 2, 3)

array([[[[[[  1.,   2.,   3.],
           [  6.,   7.,   8.],
           [ 11.,  12.,  13.]],

          [[ 26.,  27.,  28.],
           [ 31.,  32.,  33.],
           [ 36.,  37.,  38.]],

          [[ 51.,  52.,  53.],
           [ 56.,  57.,  58.],
           [ 61.,  62.,  63.]]],


         [[[  3.,   4.,   5.],
           [  8.,   9.,  10.],
           [ 13.,  14.,  15.]],

          [[ 28.,  29.,  30.],
           [ 33.,  34.,  35.],
           [ 38.,  39.,  40.]],

          [[ 53.,  54.,  55.],
           [ 58.,  59.,  60.],
           [ 63.,  64.,  65.]]]],



        [[[[ 11.,  12.,  13.],
           [ 16.,  17.,  18.],
           [ 21.,  22.,  23.]],

          [[ 36.,  37.,  38.],
           [ 41.,  42.,  43.],
           [ 46.,  47.,  48.]],

          [[ 61.,  62.,  63.],
           [ 66.,  67.,  68.],
           [ 71.,  72.,  73.]]],


         [[[ 13.,  14.,  15.],
           [ 18.,  19.,  20.],
           [ 23.,  24.,  25.]],

          [[ 38.,  39.,  40.],
           

In [54]:
r_col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
r_col.shape, r_col

((8, 27),
 array([[  1.,   2.,   3.,   6.,   7.,   8.,  11.,  12.,  13.,  26.,  27.,
          28.,  31.,  32.,  33.,  36.,  37.,  38.,  51.,  52.,  53.,  56.,
          57.,  58.,  61.,  62.,  63.],
        [  3.,   4.,   5.,   8.,   9.,  10.,  13.,  14.,  15.,  28.,  29.,
          30.,  33.,  34.,  35.,  38.,  39.,  40.,  53.,  54.,  55.,  58.,
          59.,  60.,  63.,  64.,  65.],
        [ 11.,  12.,  13.,  16.,  17.,  18.,  21.,  22.,  23.,  36.,  37.,
          38.,  41.,  42.,  43.,  46.,  47.,  48.,  61.,  62.,  63.,  66.,
          67.,  68.,  71.,  72.,  73.],
        [ 13.,  14.,  15.,  18.,  19.,  20.,  23.,  24.,  25.,  38.,  39.,
          40.,  43.,  44.,  45.,  48.,  49.,  50.,  63.,  64.,  65.,  68.,
          69.,  70.,  73.,  74.,  75.],
        [ 76.,  77.,  78.,  81.,  82.,  83.,  86.,  87.,  88., 101., 102.,
         103., 106., 107., 108., 111., 112., 113., 126., 127., 128., 131.,
         132., 133., 136., 137., 138.],
        [ 78.,  79.,  80.,  83.,  84.,  

---
# 레퍼런스 없이 연습

In [ ]:
# img_size=(1, 3, 5, 5), filter_size=(3, 3, 3), stride=1, padding=1
import numpy as np
input_data = np.arange(1, 1 + 1*3*5*5).reshape(1, 3, 5, 5)

N, C, H, W = input_data.shape
filter_h = filter_w = 3
stride = 1
padding = 1
out_h = (H - filter_h + 2*padding) // stride + 1
out_w = (W - filter_w + 2*padding) // stride + 1
img = np.pad(input_data, ((0, 0), (0, 0), (padding, padding), (padding, padding)), 'constant')
col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

for y in range(filter_h):
    y_max = y + out_h * stride
    for x in range(filter_w):
        x_max = x + out_w * stride
        col[:, :, y, x, ...] = img[:, :, y:y_max:stride, x:x_max:stride]

col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1).shape, col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)

((25, 27),
 array([[ 0.,  0.,  0.,  0.,  1.,  2.,  0.,  6.,  7.,  0.,  0.,  0.,  0.,
         26., 27.,  0., 31., 32.,  0.,  0.,  0.,  0., 51., 52.,  0., 56.,
         57.],
        [ 0.,  0.,  0.,  1.,  2.,  3.,  6.,  7.,  8.,  0.,  0.,  0., 26.,
         27., 28., 31., 32., 33.,  0.,  0.,  0., 51., 52., 53., 56., 57.,
         58.],
        [ 0.,  0.,  0.,  2.,  3.,  4.,  7.,  8.,  9.,  0.,  0.,  0., 27.,
         28., 29., 32., 33., 34.,  0.,  0.,  0., 52., 53., 54., 57., 58.,
         59.],
        [ 0.,  0.,  0.,  3.,  4.,  5.,  8.,  9., 10.,  0.,  0.,  0., 28.,
         29., 30., 33., 34., 35.,  0.,  0.,  0., 53., 54., 55., 58., 59.,
         60.],
        [ 0.,  0.,  0.,  4.,  5.,  0.,  9., 10.,  0.,  0.,  0.,  0., 29.,
         30.,  0., 34., 35.,  0.,  0.,  0.,  0., 54., 55.,  0., 59., 60.,
          0.],
        [ 0.,  1.,  2.,  0.,  6.,  7.,  0., 11., 12.,  0., 26., 27.,  0.,
         31., 32.,  0., 36., 37.,  0., 51., 52.,  0., 56., 57.,  0., 61.,
         62.],
        [ 1

In [123]:
np.pad(input_data, ((0, 0), (0, 0), (padding, padding), (padding, padding)), 'constant')

array([[[[ 0,  0,  0,  0,  0,  0,  0],
         [ 0,  1,  2,  3,  4,  5,  0],
         [ 0,  6,  7,  8,  9, 10,  0],
         [ 0, 11, 12, 13, 14, 15,  0],
         [ 0, 16, 17, 18, 19, 20,  0],
         [ 0, 21, 22, 23, 24, 25,  0],
         [ 0,  0,  0,  0,  0,  0,  0]],

        [[ 0,  0,  0,  0,  0,  0,  0],
         [ 0, 26, 27, 28, 29, 30,  0],
         [ 0, 31, 32, 33, 34, 35,  0],
         [ 0, 36, 37, 38, 39, 40,  0],
         [ 0, 41, 42, 43, 44, 45,  0],
         [ 0, 46, 47, 48, 49, 50,  0],
         [ 0,  0,  0,  0,  0,  0,  0]],

        [[ 0,  0,  0,  0,  0,  0,  0],
         [ 0, 51, 52, 53, 54, 55,  0],
         [ 0, 56, 57, 58, 59, 60,  0],
         [ 0, 61, 62, 63, 64, 65,  0],
         [ 0, 66, 67, 68, 69, 70,  0],
         [ 0, 71, 72, 73, 74, 75,  0],
         [ 0,  0,  0,  0,  0,  0,  0]]]])

---
# col = np.zeros((N, out_h, out_w, C, filter_h, filter_w)) 로 구현 해보기

In [2]:
import numpy as np
input_data = np.arange(1, 2*3*5*5 + 1).reshape(2, 3, 5, 5)
input_data

array([[[[  1,   2,   3,   4,   5],
         [  6,   7,   8,   9,  10],
         [ 11,  12,  13,  14,  15],
         [ 16,  17,  18,  19,  20],
         [ 21,  22,  23,  24,  25]],

        [[ 26,  27,  28,  29,  30],
         [ 31,  32,  33,  34,  35],
         [ 36,  37,  38,  39,  40],
         [ 41,  42,  43,  44,  45],
         [ 46,  47,  48,  49,  50]],

        [[ 51,  52,  53,  54,  55],
         [ 56,  57,  58,  59,  60],
         [ 61,  62,  63,  64,  65],
         [ 66,  67,  68,  69,  70],
         [ 71,  72,  73,  74,  75]]],


       [[[ 76,  77,  78,  79,  80],
         [ 81,  82,  83,  84,  85],
         [ 86,  87,  88,  89,  90],
         [ 91,  92,  93,  94,  95],
         [ 96,  97,  98,  99, 100]],

        [[101, 102, 103, 104, 105],
         [106, 107, 108, 109, 110],
         [111, 112, 113, 114, 115],
         [116, 117, 118, 119, 120],
         [121, 122, 123, 124, 125]],

        [[126, 127, 128, 129, 130],
         [131, 132, 133, 134, 135],
         [136, 1

In [128]:
input_data = np.arange(1, 2*3*5*5 + 1).reshape(2, 3, 5, 5)

N, C, H, W = input_data.shape
filter_h = filter_w = 3
pad = 0
stride = 2
out_h = (H + 2*pad - filter_h)//stride + 1 # 3
out_w = (W + 2*pad - filter_w)//stride + 1 # 3

img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
col = np.zeros((N, out_h, out_w, C, filter_h, filter_w))

for y in range(out_h):
    y_start = y * stride
    y_end = y_start + filter_h
    for x in range(out_w):
        x_start = x * stride
        x_end = x_start + filter_w
        col[:, y, x, :, :, :] = img[:, :, y_start:y_end, x_start:x_end]
        break

col[0, 0]

array([[[[ 1.,  2.,  3.],
         [ 6.,  7.,  8.],
         [11., 12., 13.]],

        [[26., 27., 28.],
         [31., 32., 33.],
         [36., 37., 38.]],

        [[51., 52., 53.],
         [56., 57., 58.],
         [61., 62., 63.]]],


       [[[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]],

        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]],

        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]]]])

In [52]:
input_data = np.arange(1, 2*3*5*5 + 1).reshape(2, 3, 5, 5)

N, C, H, W = input_data.shape
filter_h = filter_w = 3
pad = 0
stride = 2
out_h = (H + 2*pad - filter_h)//stride + 1 # 3
out_w = (W + 2*pad - filter_w)//stride + 1 # 3

img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
col = np.zeros((N, out_h, out_w, C, filter_h, filter_w))

for y in range(out_h):
    y_start = y * stride
    y_end = y_start + filter_h
    for x in range(out_w):
        x_start = x * stride
        x_end = x_start + filter_w
        col[:, y, x, :, :, :] = img[:, :, y_start:y_end, x_start:x_end]
col

array([[[[[[  1.,   2.,   3.],
           [  6.,   7.,   8.],
           [ 11.,  12.,  13.]],

          [[ 26.,  27.,  28.],
           [ 31.,  32.,  33.],
           [ 36.,  37.,  38.]],

          [[ 51.,  52.,  53.],
           [ 56.,  57.,  58.],
           [ 61.,  62.,  63.]]],


         [[[  3.,   4.,   5.],
           [  8.,   9.,  10.],
           [ 13.,  14.,  15.]],

          [[ 28.,  29.,  30.],
           [ 33.,  34.,  35.],
           [ 38.,  39.,  40.]],

          [[ 53.,  54.,  55.],
           [ 58.,  59.,  60.],
           [ 63.,  64.,  65.]]]],



        [[[[ 11.,  12.,  13.],
           [ 16.,  17.,  18.],
           [ 21.,  22.,  23.]],

          [[ 36.,  37.,  38.],
           [ 41.,  42.,  43.],
           [ 46.,  47.,  48.]],

          [[ 61.,  62.,  63.],
           [ 66.,  67.,  68.],
           [ 71.,  72.,  73.]]],


         [[[ 13.,  14.,  15.],
           [ 18.,  19.,  20.],
           [ 23.,  24.,  25.]],

          [[ 38.,  39.,  40.],
           

In [53]:
col.reshape(N*out_h*out_w, -1)

array([[  1.,   2.,   3.,   6.,   7.,   8.,  11.,  12.,  13.,  26.,  27.,
         28.,  31.,  32.,  33.,  36.,  37.,  38.,  51.,  52.,  53.,  56.,
         57.,  58.,  61.,  62.,  63.],
       [  3.,   4.,   5.,   8.,   9.,  10.,  13.,  14.,  15.,  28.,  29.,
         30.,  33.,  34.,  35.,  38.,  39.,  40.,  53.,  54.,  55.,  58.,
         59.,  60.,  63.,  64.,  65.],
       [ 11.,  12.,  13.,  16.,  17.,  18.,  21.,  22.,  23.,  36.,  37.,
         38.,  41.,  42.,  43.,  46.,  47.,  48.,  61.,  62.,  63.,  66.,
         67.,  68.,  71.,  72.,  73.],
       [ 13.,  14.,  15.,  18.,  19.,  20.,  23.,  24.,  25.,  38.,  39.,
         40.,  43.,  44.,  45.,  48.,  49.,  50.,  63.,  64.,  65.,  68.,
         69.,  70.,  73.,  74.,  75.],
       [ 76.,  77.,  78.,  81.,  82.,  83.,  86.,  87.,  88., 101., 102.,
        103., 106., 107., 108., 111., 112., 113., 126., 127., 128., 131.,
        132., 133., 136., 137., 138.],
       [ 78.,  79.,  80.,  83.,  84.,  85.,  88.,  89.,  90., 103

In [54]:
# 위와 비교 (im2col 원본)
input_data = np.arange(1, 2*3*5*5 + 1).reshape(2, 3, 5, 5)

N, C, H, W = input_data.shape
filter_h = filter_w = 3
pad = 0
stride = 2
out_h = (H + 2*pad - filter_h)//stride + 1
out_w = (W + 2*pad - filter_w)//stride + 1

img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

for y in range(filter_h):
    y_max = y + stride*out_h
    for x in range(filter_w):
        x_max = x + stride*out_w
        col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)

array([[  1.,   2.,   3.,   6.,   7.,   8.,  11.,  12.,  13.,  26.,  27.,
         28.,  31.,  32.,  33.,  36.,  37.,  38.,  51.,  52.,  53.,  56.,
         57.,  58.,  61.,  62.,  63.],
       [  3.,   4.,   5.,   8.,   9.,  10.,  13.,  14.,  15.,  28.,  29.,
         30.,  33.,  34.,  35.,  38.,  39.,  40.,  53.,  54.,  55.,  58.,
         59.,  60.,  63.,  64.,  65.],
       [ 11.,  12.,  13.,  16.,  17.,  18.,  21.,  22.,  23.,  36.,  37.,
         38.,  41.,  42.,  43.,  46.,  47.,  48.,  61.,  62.,  63.,  66.,
         67.,  68.,  71.,  72.,  73.],
       [ 13.,  14.,  15.,  18.,  19.,  20.,  23.,  24.,  25.,  38.,  39.,
         40.,  43.,  44.,  45.,  48.,  49.,  50.,  63.,  64.,  65.,  68.,
         69.,  70.,  73.,  74.,  75.],
       [ 76.,  77.,  78.,  81.,  82.,  83.,  86.,  87.,  88., 101., 102.,
        103., 106., 107., 108., 111., 112., 113., 126., 127., 128., 131.,
        132., 133., 136., 137., 138.],
       [ 78.,  79.,  80.,  83.,  84.,  85.,  88.,  89.,  90., 103

### col = np.zeros((N, out_h, out_w, C, filter_h, filter_w))로 안 하는 이유
**루프 횟수 차이**

img_size = 224, filter_size = (3, 3), stride = 2, padding = 0, output_size = (224 - 3 ) // 2 + 1 = 111

- 전통 방식: fh × fw = 9번 루프 (작음)
<br>    → 안쪽에서 NumPy 슬라이싱 하나로 out_h × out_w × N × C 만큼 동시에 복사 (벡터화 극대화)

- output_size로 회전 방식: out_h × out_w 번 루프 (보통 수천~수만 번, 예: 111×111 출력이면 12,321 번)
<br>    → Python 루프 오버헤드 누적 + 각 슬라이싱이 작아서 벡터화 이득 적음

transpose는 자료구조 변경 없이 포인터만 변경되는 뷰
<br>
더 정확히는 “데이터 복사 없이 shape과 strides만 바꾼 뷰”

# col2im 구현하기

In [ ]:
def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
    """(im2col과 반대) 2차원 배열을 입력받아 다수의 이미지 묶음으로 변환한다.
    
    Parameters
    ----------
    col : 2차원 배열(입력 데이터)
    input_shape : 원래 이미지 데이터의 형상（예：(10, 1, 28, 28)）
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    img : 변환된 이미지들
    """
    N, C, H, W = input_shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

    img = np.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

    return img[:, :, pad:H + pad, pad:W + pad]

In [ ]:
input_shape = (1, 2, 4, 4)
filter_h = filter_w = 3
stride=1
pad=0

N, C, H, W = input_shape
col = np.arange(1, 1 + N*out_h*out_w*C*filter_h*filter_w)
out_h = (H + 2*pad - filter_h)//stride + 1
out_w = (W + 2*pad - filter_w)//stride + 1
col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

img = np.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))
for y in range(filter_h):
    y_max = y + stride*out_h
    for x in range(filter_w):
        x_max = x + stride*out_w
        img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

img[:, :, pad:H + pad, pad:W + pad]

array([[[[  1.,  21.,  23.,  21.],
         [ 41., 120., 124.,  81.],
         [ 47., 132., 136.,  87.],
         [ 43., 105., 107.,  63.]],

        [[ 10.,  39.,  41.,  30.],
         [ 59., 156., 160.,  99.],
         [ 65., 168., 172., 105.],
         [ 52., 123., 125.,  72.]]]])